In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("C:/Data/uber_rides_data.xlsx - sample_train.csv")

#### What is the shape of given dataset

In [3]:
df.shape

(200000, 8)

#### How many integer columns(by default) are given in the dataset?
#### What is the data type of ' pickup_datetime' feature in your data?*

In [4]:
df.dtypes

ride_id                int64
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

#### How many missing values exists in 'dropoff_longitude' column?

In [5]:
df.dropoff_longitude.isnull().sum()

1

#### What is the average fare amount?
#### Remove the null values from the dataframe to answer the following question

In [6]:
average_fare_amount = df['fare_amount'].mean()

print("Average Fare Amount:", average_fare_amount)

Average Fare Amount: 11.359955250000002


In [7]:
df_clean = df.dropna()

In [8]:
average_fare_amount = df_clean['fare_amount'].mean()

print("Average Fare Amount:", average_fare_amount)

Average Fare Amount: 11.359891549457748


#### Which of the following is the correct syntax to convert 'pickup_datetime' to datetime datatype?

In [ ]:
pd.to_datetime(df['pickup_datetime'])

#### Calculate distance between each pickup and dropoff points using Haversine formula. 
#### What is the median haversine distance between pickup and dropoff location according to the given dataset?


In [9]:
from math import radians, sin, cos, sqrt, atan2

In [10]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    radius_of_earth = 6371  # Earth's radius in kilometers
    distance = radius_of_earth * c

    return distance

In [11]:
df['haversine_distance'] = df.apply(lambda row: haversine_distance(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)


In [12]:
median_haversine_distance = df['haversine_distance'].median()
print("Median Haversine Distance:", median_haversine_distance)


Median Haversine Distance: 2.1209923961833708


#### What is the maximum haversine distance between pickup and dropoff location according to the given dataset?

In [13]:
maximum_haversine_distance = df['haversine_distance'].max()
print("Maximum Haversine Distance:", maximum_haversine_distance)

Maximum Haversine Distance: 16409.239135313168


#### How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset

In [14]:
rides_with_zero_distance = df[df['haversine_distance'] == 0.0]
num_rides_with_zero_distance = len(rides_with_zero_distance)

print("Number of rides with 0.0 Haversine distance:", num_rides_with_zero_distance)

Number of rides with 0.0 Haversine distance: 5632


#### What is the mean 'fare_amount' for rides with 0 haversine distance?*
Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.


In [15]:
mean_fare_for_zero_distance = rides_with_zero_distance['fare_amount'].mean()
print("Mean fare amount for rides with 0.0 Haversine distance:", mean_fare_for_zero_distance)

Mean fare amount for rides with 0.0 Haversine distance: 11.585317826704546


In [16]:
# Haversine distance of 0.0 between pickup and dropoff locations typically indicates that 
# the pickup and dropoff points are the same or very close together. 
# Moreover Uber might be having a fixed ammount for every ride irrespective of distance.
# the price of a ride extends this fixed ammount further by ammount of distance covered.

#### What is the maximum 'fare_amount' for a ride?

In [17]:
maximum_fare_amount = df['fare_amount'].max()
print("Maximum fare amount for a ride:", maximum_fare_amount)

Maximum fare amount for a ride: 499.0


#### What is the haversine distance between pickup and dropoff location for the costliest ride?*
Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.

In [18]:
costliest_ride = df[df['fare_amount'] == maximum_fare_amount].iloc[0]

haversine_distance_costliest = haversine_distance(
    costliest_ride['pickup_latitude'],
    costliest_ride['pickup_longitude'],
    costliest_ride['dropoff_latitude'],
    costliest_ride['dropoff_longitude']
)

print("Haversine distance for the costliest ride:", haversine_distance_costliest)


Haversine distance for the costliest ride: 0.0007899213191009994


In [19]:
# High fare amount for a ride could be indicative of a data issue or an outlier. 
# It's possible that such a high fare amount could be due to data entry errors, anomalies, or even fraudulent activities..

#### How many rides were recorded in the year 2014

In [20]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

df['pickup_year'] = df['pickup_datetime'].dt.year

rides_in_2014 = len(df[df['pickup_year'] == 2014])

print("Number of rides recorded in the year 2014:", rides_in_2014)


Number of rides recorded in the year 2014: 29968


#### How many rides were recorded in the first quarter of 2014

In [21]:
df['pickup_quarter'] = df['pickup_datetime'].dt.quarter

rides_in_q1_2014 = len(df[(df['pickup_year'] == 2014) & (df['pickup_quarter'] == 1)])

print("Number of rides recorded in the first quarter of 2014:", rides_in_q1_2014)

Number of rides recorded in the first quarter of 2014: 7687


#### On which day of the week in September 2010, maximum rides were recorded 

In [22]:
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek
df['year_month'] = df['pickup_datetime'].dt.strftime('%Y-%m')

rides_in_september_2010 = df[df['year_month'] == '2010-09']

rides_by_day_of_week = rides_in_september_2010['day_of_week'].value_counts()

day_with_max_rides = rides_by_day_of_week.idxmax()

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
max_rides_day_name = days_of_week[day_with_max_rides]

print("Day of the week in September 2010 with the maximum rides:", max_rides_day_name)

Day of the week in September 2010 with the maximum rides: Thursday


#### Apply a Machine Learning Algorithm to predict the fare amount given following input features:
passenger_count, distance and ride_week_day.

Perform a 70-30 split of data.

Which algorithm gives the least adjusted R square value?*


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [24]:
df.dropna(inplace=True)

X = df[['passenger_count', 'haversine_distance', 'day_of_week']]
y = df['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
X_train.head()

passenger_count  haversine_distance  day_of_week
194320                1            1.270348            3
774                   2            6.406598            5
75676                 1            3.016657            2
37127                 1            1.546083            4
96757                 5            2.492464            6

In [26]:
y_train.head()

194320     6.1
774       16.0
75676     13.5
37127      7.5
96757      8.1
Name: fare_amount, dtype: float64

In [27]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 10.138402052152694


In [28]:
model2 = RandomForestRegressor()

model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 6.169777898467877
